In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap
# !{sys.executable} -m pip install pip rasterstats 

In [4]:
import ee
# ee.Authenticate()

In [5]:
ee.Initialize()

In [6]:
import numpy as np
import requests
import os
import pandas as pd
import rasterio
import boto3
import geopandas as gpd
import io
# from rasterstats import zonal_stats
import fiona
import rasterio.mask
import geemap
import glob
import boto3

In [7]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# Read input data

In [8]:
# define directory
out_dir = os.getcwd()
aws_s3_dir = "https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data"

In [9]:
## create map
Map = geemap.Map(height="350px")
# Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [10]:
# read esa land cover
esa_land_cover = ee.ImageCollection('ESA/WorldCover/v100').first()
esa_PROJ = esa_land_cover.projection().crs().getInfo()
esaScale = esa_land_cover.projection().nominalScale()
esaScale20 = esa_land_cover.projection().atScale(20).nominalScale()
builtup = esa_land_cover.eq(50)
# Map.addLayer(builtup,{},"builtup areas")

In [11]:
## Calculate and load vegetation cover raster

NDVIthreshold = 0.4 # decimal
year = 2020

yearStr = str(year)
startdate = ''+yearStr+'-01-01'
enddate = ''+yearStr+'-12-31'

s2 = ee.ImageCollection("COPERNICUS/S2")

def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)

green = s2.filterDate(startdate, enddate).map(addNDVI)
green = green.qualityMosaic('NDVI').select('NDVI').float();
#green = green.addBands(ee.Image(year).rename('time_start'))
greenmask = green.updateMask(green.select('NDVI').gte(NDVIthreshold))
# greenmask = greenmask.reproject(**{'crs':esa_PROJ,'scale':esaScale})##'crsTransform':esa_PROJ.transform,
# Map.addLayer(greenmask,{},"green areas")

builtgreenmask = greenmask.updateMask(builtup)
# Map.addLayer(builtgreenmask,{},"green builtup areas")

In [12]:
# get list of c4f cities
boundary_georef = pd.read_csv('https://cities-cities4forests.s3.eu-west-3.amazonaws.com/data/boundaries/v_0/boundary_georef.csv')
boundary_georef

,city_name,geo_name,aoi_boundary_name,units_boundary_name,city_boundary_name,country_code,geo_level
0,Salvador,BRA-Salvador,ADM4union,ADM4,BRA-Salvador-ADM4,BRA,ADM4
1,Bukavu,COD-Bukavu,ADM3union,ADM3,COD-Bukavu-ADM3,COD,ADM3
2,Uvira,COD-Uvira,ADM3union,ADM3,COD-Uvira-ADM3,COD,ADM3
3,Brazzaville,COG-Brazzaville,ADM4union,ADM4,COG-Brazzaville-ADM4,COG,ADM4
4,Barranquilla,COL-Barranquilla,ADM4union,ADM4,COL-Barranquilla-ADM4,COL,ADM4
5,Addis_Ababa,ETH-Addis_Ababa,ADM4union,ADM4,ETH-Addis_Ababa-ADM4,ETH,ADM4
6,Dire_Dawa,ETH-Dire_Dawa,ADM3union,ADM3,ETH-Dire_Dawa-ADM3,ETH,ADM3
7,Nairobi,KEN-Nairobi,ADM3union,ADM3,KEN-Nairobi-ADM3,KEN,ADM3
8,Antananarivo,MDG-Antananarivo,ADM4union,ADM4,MDG-Antananarivo-ADM4,MDG,ADM4
9,Mexico_City,MEX-Mexico_City,ADM2union,ADM2,MEX-Mexico_City-ADM2,MEX,ADM2


# Compute indicator

In [19]:
this_indicatorDF = pd.DataFrame() 
this_indicator = ee.FeatureCollection([])
this_indicator

In [20]:

# define calcuation function to get pixel counts, convert to percents and append to data frame
def CountCalcsDF(FC,DF):
    # reduce images to get vegetation and built-up pixel counts
    pixelcounts = builtgreenmask.reduceRegions(FC,ee.Reducer.count().setOutputs(['BuiltGreenPixels']),esaScale)
    pixelcounts = builtup.reduceRegions(pixelcounts,ee.Reducer.count().setOutputs(['BuiltPixels']),esaScale)

    # convert pixel counts to area percentages and saves to FC as property
    def toPct(feat):
        pct = ee.Number(1).subtract((feat.getNumber('BuiltGreenPixels')).divide(feat.getNumber('BuiltPixels')))
        return feat.set({
            'PctBuiltwoGreen_'+yearStr+'': pct
      })

    pixelcounts = pixelcounts.map(toPct).select(['geo_id','PctBuiltwoGreen_'+yearStr+''])

    # store in df and append
    df = geemap.ee_to_pandas(pixelcounts)
    df = df.rename(columns={'PctBuiltwoGreen_'+yearStr+'': 'GRE_4_5_percentBuiltupWOvegetationcover'+yearStr+''})
    DF = DF.append(df)
    return DF


In [21]:
# use if using EE featurecollections for calculations
# define calcuation function to get pixel counts, convert to percents and append to data frame
def CountCalcsEE(FC,DF):
    # reduce images to get vegetation and built-up pixel counts
    pixelcounts = builtgreenmask.reduceRegions(FC,ee.Reducer.count().setOutputs(['BuiltGreenPixels']),esaScale)
    pixelcounts = builtup.reduceRegions(pixelcounts,ee.Reducer.count().setOutputs(['BuiltPixels']),esaScale)

    # convert pixel counts to area percentages and saves to FC as property
    def toPct(feat):
        pct = ee.Number(1).subtract((feat.getNumber('BuiltGreenPixels')).divide(feat.getNumber('BuiltPixels')))
        return feat.set({
            'PctBuiltwoGreen_'+yearStr+'': pct
      })

    pixelcounts = pixelcounts.map(toPct).select(['geo_id','PctBuiltwoGreen_'+yearStr+''])
    
    # amend existing FeatureCollection with pixel counts for new geographies
    DF = ee.FeatureCollection([DF,pixelcounts]).flatten()
    return DF


In [56]:
for i in range(11,len(boundary_georef)): #9,10):# for Mexico City
    print(i)
    geo_name = boundary_georef.loc[i, 'geo_name']
    print("\n geo_name: "+geo_name)
    
    boundary_id_aoi = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'aoi_boundary_name']
    boundary_id_unit = boundary_georef.loc[i, 'geo_name']+'-'+boundary_georef.loc[i, 'units_boundary_name']

    
    # process aoi level ------
    print("\n boundary_id_aoi: "+boundary_id_aoi)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_aoi+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicatorDF = CountCalcsDF(boundary_geo_ee,this_indicatorDF)
    # this_indicator = CountCalcsEE(boundary_geo_ee,this_indicator) # run this instead if using CountCalcsEE approach


    # process unit of analysis level ------
    print("\n boundary_id_unit: "+boundary_id_unit)
    # read boundaries
    boundary_path = aws_s3_dir +'/boundaries/v_0/boundary-'+boundary_id_unit+'.geojson'
    boundary_geo = requests.get(boundary_path).json()
    boundary_geo_ee = geemap.geojson_to_ee(boundary_geo)
    this_indicatorDF = CountCalcsDF(boundary_geo_ee,this_indicatorDF)
    # this_indicator = CountCalcsEE(boundary_geo_ee,this_indicator) # run this instead if using CountCalcsEE approach


11

 geo_name: RWA-Musanze

 boundary_id_aoi: RWA-Musanze-ADM5union

 boundary_id_unit: RWA-Musanze-ADM5


In [58]:
this_indicatorDF

,GRE_4_5_percentBuiltupWOvegetationcover2020,geo_id
0,0.956027,MEX-Mexico_City_ADM2-union_1
1,0.919496,MEX-Mexico_City_ADM2_1
2,0.900134,MEX-Mexico_City_ADM2_2
3,0.948390,MEX-Mexico_City_ADM2_3
4,0.950362,MEX-Mexico_City_ADM2_4
...,...,...
135,0.984369,RWA-Musanze_ADM5_136
136,1.000000,RWA-Musanze_ADM5_137
137,1.000000,RWA-Musanze_ADM5_138
138,1.000000,RWA-Musanze_ADM5_139


# Workaround for timeout problems for specific geographies

In [43]:
    # if timeout problems for a geography, use CountCalcsEE and save ee.FeatureCollection as EE asset before coverting to dataframe
    task = ee.batch.Export.table.toAsset(
        collection = this_indicator, 
        description = 'thisindicator',
        assetId = 'users/emackres/thisindicator',
    )

    task.start()

In [51]:
    task.status()

{'state': 'COMPLETED',
 'description': 'thisindicator',
 'creation_timestamp_ms': 1663869991306,
 'update_timestamp_ms': 1663871016391,
 'start_timestamp_ms': 1663870002711,
 'task_type': 'EXPORT_FEATURES',
 'destination_uris': ['https://code.earthengine.google.com/?asset=projects/earthengine-legacy/assets/users/emackres/thisindicator'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 121150.4296875,
 'id': 'DD6EW2J4XDTKIMPFJHCWMRNV',
 'name': 'projects/earthengine-legacy/operations/DD6EW2J4XDTKIMPFJHCWMRNV'}

In [52]:
    # wait until EE asset is generated (with task state of "COMPLETED") before running
    # store FC in df and apend to this_indicatorDF

    FC = ee.FeatureCollection('users/emackres/thisindicator')
    df = geemap.ee_to_pandas(FC)
    df = df.rename(columns={'PctBuiltwoGreen_'+yearStr+'': 'GRE_4_5_percentBuiltupWOvegetationcover'+yearStr+''})
    this_indicatorDF = this_indicatorDF.append(df)


In [53]:
    this_indicatorDF

,GRE_4_5_percentBuiltupWOvegetationcover2020,geo_id
0,0.956027,MEX-Mexico_City_ADM2-union_1
1,0.919496,MEX-Mexico_City_ADM2_1
2,0.900134,MEX-Mexico_City_ADM2_2
3,0.948390,MEX-Mexico_City_ADM2_3
4,0.950362,MEX-Mexico_City_ADM2_4
...,...,...
91,0.995174,MEX-Monterrey_ADM2_14
92,0.924334,MEX-Monterrey_ADM2_15
93,0.985204,MEX-Monterrey_ADM2_16
94,0.994309,MEX-Monterrey_ADM2_17


In [42]:
    #delete GEE asset 
    ee.data.deleteAsset('users/emackres/thisindicator')

# Merge with indicator table

In [59]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir + '/indicators/cities_indicators_v2test.csv') 
cities_indicators#.head()

,geo_id,geo_level,geo_name,geo_parent_name,percent_of_tree_cover,GRE_3_1_percentOpenSpaceinBuiltup,GRE_3_2_percentPopwOpenSpaceAccess,GRE_3_3_percentPopwTreeCoverAcess,GRE_4_1_percentFloodProneinBuiltup2050,GRE_4_5_percentBuiltupWOvegetationcover2020,GRE_4_2_percentChangeinMaxDailyPrecip2020to2050,GRE_4_3_percentBuiltupWithin1mAboveDrainage,GRE_4_4_percentImperviousinBuiltup2018,GRE_1_1_percentChangeinDaysAbove35C2020to2050,GRE_4_7_percentSteepSlopesWOvegetationcover2020,GRE_4_6_percentRiparianZonewoVegorWatercover2020
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,NaN,0.043743,0.743757,0.004452,0.228568,0.930463,-0.023907,0.028730,0.355911,0.084367,0.365891,0.443769
1,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,NaN,0.072329,0.990298,0.000000,0.003237,0.951636,NaN,0.007564,0.184946,NaN,0.102096,0.182949
2,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,NaN,0.103440,0.984178,0.000000,0.000000,0.915644,NaN,0.010563,0.170100,NaN,0.105796,0.291383
3,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,NaN,0.050375,0.936113,0.000000,0.000000,0.884508,NaN,0.063145,0.332444,NaN,0.241692,0.490249
4,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador,NaN,0.048150,0.945183,0.000000,0.277053,0.945971,NaN,0.076025,0.610676,NaN,0.537859,0.735183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,RWA-Musanze_ADM5_136,ADM5,Burengo,RWA-Musanze,28.526825,0.000000,0.000000,0.953596,0.000000,0.984369,NaN,0.011839,0.001387,NaN,0.000000,0.031915
1078,RWA-Musanze_ADM5_137,ADM5,Bwamazi,RWA-Musanze,NaN,0.000000,0.000000,0.000000,0.000000,1.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN
1079,RWA-Musanze_ADM5_138,ADM5,Kadahenda,RWA-Musanze,NaN,0.000000,0.000000,0.000000,0.000000,1.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN
1080,RWA-Musanze_ADM5_139,ADM5,Karwesero,RWA-Musanze,NaN,0.000000,0.000000,0.000000,0.000000,1.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN


In [60]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    else:
        print("add new indicators")
        new_indicator_table = new_indicator_table.drop_duplicates()
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left',
                                                     validate='one_to_many')
    return(cities_indicators_df)

In [61]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = this_indicatorDF,
                                            indicator_name = 'GRE_4_5_percentBuiltupWOvegetationcover'+yearStr+'')

replace with new calculations


In [62]:
cities_indicators_merged

,geo_id,geo_level,geo_name,geo_parent_name,percent_of_tree_cover,GRE_3_1_percentOpenSpaceinBuiltup,GRE_3_2_percentPopwOpenSpaceAccess,GRE_3_3_percentPopwTreeCoverAcess,GRE_4_1_percentFloodProneinBuiltup2050,GRE_4_2_percentChangeinMaxDailyPrecip2020to2050,GRE_4_3_percentBuiltupWithin1mAboveDrainage,GRE_4_4_percentImperviousinBuiltup2018,GRE_1_1_percentChangeinDaysAbove35C2020to2050,GRE_4_7_percentSteepSlopesWOvegetationcover2020,GRE_4_6_percentRiparianZonewoVegorWatercover2020,GRE_4_5_percentBuiltupWOvegetationcover2020
0,BRA-Salvador_ADM4-union_1,ADM4-union,BRA-Salvador,BRA-Salvador,NaN,0.043743,0.743757,0.004452,0.228568,-0.023907,0.028730,0.355911,0.084367,0.365891,0.443769,0.930463
1,BRA-Salvador_ADM4_1,ADM4,Pituaçu,BRA-Salvador,NaN,0.072329,0.990298,0.000000,0.003237,NaN,0.007564,0.184946,NaN,0.102096,0.182949,0.951622
2,BRA-Salvador_ADM4_2,ADM4,Patamares,BRA-Salvador,NaN,0.103440,0.984178,0.000000,0.000000,NaN,0.010563,0.170100,NaN,0.105796,0.291383,0.915680
3,BRA-Salvador_ADM4_3,ADM4,Piatã,BRA-Salvador,NaN,0.050375,0.936113,0.000000,0.000000,NaN,0.063145,0.332444,NaN,0.241692,0.490249,0.884494
4,BRA-Salvador_ADM4_4,ADM4,Boca do Rio,BRA-Salvador,NaN,0.048150,0.945183,0.000000,0.277053,NaN,0.076025,0.610676,NaN,0.537859,0.735183,0.945971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,RWA-Musanze_ADM5_136,ADM5,Burengo,RWA-Musanze,28.526825,0.000000,0.000000,0.953596,0.000000,NaN,0.011839,0.001387,NaN,0.000000,0.031915,0.984369
1078,RWA-Musanze_ADM5_137,ADM5,Bwamazi,RWA-Musanze,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN,1.000000
1079,RWA-Musanze_ADM5_138,ADM5,Kadahenda,RWA-Musanze,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN,1.000000
1080,RWA-Musanze_ADM5_139,ADM5,Karwesero,RWA-Musanze,NaN,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,NaN,0.000000,NaN,1.000000


# Upload in aws s3

In [63]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [64]:
# upload to aws
key_data = 'data/indicators/cities_indicators_v2test.csv'
bucket_name = 'cities-cities4forests' 
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [65]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')